In [ ]:
!pip install -q spacy
!python -m spacy download en_core_web_sm
!pip install -q transformers torch
!pip -q install sacremoses


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip -q install sacremoses


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 21.6 MB/s eta 0:00:00


# Language detection (Bangla vs English)

In [ ]:
import re

BN_RANGE = re.compile(r'[\u0980-\u09FF]')

def detect_lang(query: str) -> str:
    q = query.strip()
    if BN_RANGE.search(q):
        return "bn"
    # fallback: assume english if mostly latin
    return "en"


# Normalization (required)

In [ ]:
def normalize_query(query: str, lang: str) -> str:
    q = " ".join(query.strip().split())  # collapse whitespace
    if lang == "en":
        q = q.lower()
    # For Bangla, lowercasing doesn't matter; keep as-is
    return q


# Query translation / conversion

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

class OpusTranslator:
    """
    bn -> en : Helsinki-NLP/opus-mt-bn-en
    en -> bn : Helsinki-NLP/opus-mt-en-iir with required >>ben<< token
    """
    def __init__(self):
        self.cache = {}

    def _load(self, model_name: str):
        if model_name not in self.cache:
            tok = MarianTokenizer.from_pretrained(model_name)
            mdl = MarianMTModel.from_pretrained(model_name)
            self.cache[model_name] = (tok, mdl)
        return self.cache[model_name]

    def translate_bn_to_en(self, text: str) -> str:
        model_name = "Helsinki-NLP/opus-mt-bn-en"
        tok, mdl = self._load(model_name)
        batch = tok([text], return_tensors="pt", truncation=True)
        gen = mdl.generate(**batch, max_length=64)
        return tok.decode(gen[0], skip_special_tokens=True)

    def translate_en_to_bn(self, text: str) -> str:
        # multilingual target group; needs language token >>ben<<
        model_name = "Helsinki-NLP/opus-mt-en-iir"
        tok, mdl = self._load(model_name)

        text_with_lang = f">>ben<< {text}"  # REQUIRED by model card :contentReference[oaicite:2]{index=2}
        batch = tok([text_with_lang], return_tensors="pt", truncation=True)
        gen = mdl.generate(**batch, max_length=64)
        out = tok.decode(gen[0], skip_special_tokens=True)

        # sometimes the output may keep the token; strip defensively
        return out.replace(">>ben<<", "").strip()

translator = OpusTranslator()

def translate_query(q_norm: str, lang: str):
    if lang == "bn":
        return q_norm, translator.translate_bn_to_en(q_norm)   # (bn, en)
    else:
        return q_norm, translator.translate_en_to_bn(q_norm)   # (en, bn)


# Query expansion

In [ ]:
EXPAND_DICT_EN = {
    "election": ["vote", "poll", "ballot"],
    "education": ["school", "university", "student"],
}

def expand_query(q: str, lang: str):
    terms = q.split()
    extra = []
    if lang == "en":
        for t in terms:
            extra += EXPAND_DICT_EN.get(t, [])
    # For bn you can keep empty initially
    extra = list(dict.fromkeys(extra))  # unique preserve order
    return extra


# Named Entity extraction + mapping

`Layer 1 `

In [ ]:
NE_MAP_EN2BN = {
    "bangladesh": "বাংলাদেশ",
    "dhaka": "ঢাকা",
    "sheikh hasina": "শেখ হাসিনা",
}
NE_MAP_BN2EN = {v: k for k, v in NE_MAP_EN2BN.items()}

def map_named_entities(q: str, lang: str):
    q_low = q.lower()
    mapped = []
    if lang == "en":
        for k, v in NE_MAP_EN2BN.items():
            if k in q_low:
                mapped.append((k, v))
    else:
        for k, v in NE_MAP_BN2EN.items():
            if k in q:
                mapped.append((k, v))
    return mapped


`Layer 2 `

In [ ]:
import spacy

nlp_en = spacy.load("en_core_web_sm")

def extract_ner_en(text: str):
    doc = nlp_en(text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "label": ent.label_  # PERSON, GPE, ORG, LOC, etc.
        })
    return entities


In [ ]:
from transformers import pipeline

ner_bn = pipeline(
    "ner",
    model="Davlan/xlm-roberta-base-ner-hrl",
    aggregation_strategy="simple"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def extract_ner_bn(text: str):
    results = ner_bn(text)
    entities = []
    for r in results:
        entities.append({
            "text": r["word"],
            "label": r["entity_group"]  # PER, LOC, ORG
        })
    return entities


In [ ]:
NE_MAP_EN2BN = {
    "bangladesh": "বাংলাদেশ",
    "dhaka": "ঢাকা",
    "sheikh hasina": "শেখ হাসিনা",
    "awami league": "আওয়ামী লীগ",
}

NE_MAP_BN2EN = {v: k for k, v in NE_MAP_EN2BN.items()}
def map_entities(entities, lang):
    mapped = []
    for e in entities:
        key = e["text"].lower()
        if lang == "en" and key in NE_MAP_EN2BN:
            mapped.append({
                "src": e["text"],
                "tgt": NE_MAP_EN2BN[key],
                "type": e["label"]
            })
        elif lang == "bn" and e["text"] in NE_MAP_BN2EN:
            mapped.append({
                "src": e["text"],
                "tgt": NE_MAP_BN2EN[e["text"]],
                "type": e["label"]
            })
    return mapped
def extract_and_map_entities(query: str, lang: str):
    if lang == "en":
        ents = extract_ner_en(query)
    else:
        ents = extract_ner_bn(query)

    mapped = map_entities(ents, lang)

    return {
        "entities": ents,
        "mapped_entities": mapped
    }


# Pipeline

In [ ]:
import time
def process_query(query: str):
    t0 = time.time()

    lang = detect_lang(query)
    q_norm = normalize_query(query, lang)

    t_trans0 = time.time()
    q_src, q_other = translate_query(q_norm, lang)
    trans_ms = int((time.time() - t_trans0) * 1000)

    if lang == "bn":
        q_bn, q_en = q_src, q_other
    else:
        q_en, q_bn = q_src, q_other

    t_ner0 = time.time()
    ner_info = extract_and_map_entities(q_norm, lang)
    ner_ms = int((time.time() - t_ner0) * 1000)

    expansions = expand_query(q_norm, lang)

    total_ms = int((time.time() - t0) * 1000)

    return {
        "query": query,
        "detected_lang": lang,
        "normalized": q_norm,
        "q_en": q_en,
        "q_bn": q_bn,
        "expansions": expansions,
        "entities": ner_info.get("entities", []),
        "mapped_entities": ner_info.get("mapped_entities", []),
        "time_ms": {
            "total": total_ms,
            "translation": trans_ms,
            "ner": ner_ms
        }
    }


In [ ]:
process_query("বাংলাদেশ নির্বাচন")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/309M [00:00<?, ?B/s]

{'query': 'বাংলাদেশ নির্বাচন',
 'detected_lang': 'bn',
 'normalized': 'বাংলাদেশ নির্বাচন',
 'q_en': 'Bangladesh selection',
 'q_bn': 'বাংলাদেশ নির্বাচন',
 'expansions': [],
 'entities': [{'text': 'বাংলাদেশ', 'label': 'LOC'}],
 'mapped_entities': [{'src': 'বাংলাদেশ', 'tgt': 'bangladesh', 'type': 'LOC'}],
 'time_ms': {'total': 8300, 'translation': 8087, 'ner': 213}}

In [ ]:

process_query("Bangladesh election")

{'query': 'Bangladesh election',
 'detected_lang': 'en',
 'normalized': 'bangladesh election',
 'q_en': 'bangladesh election',
 'q_bn': 'নিষেধাজ্ঞা নির্বাচন',
 'expansions': ['vote', 'poll', 'ballot'],
 'entities': [],
 'mapped_entities': [],
 'time_ms': {'total': 376, 'translation': 359, 'ner': 16}}

In [ ]:
queries = [
    "বাংলাদেশ নির্বাচন",
    "Bangladesh election",
    "ঢাকা traffic problem",
    "বাংলাদেশ election results",
    "চেয়ার"
]

for q in queries:
    print(process_query(q))
    print("-" * 80)


{'query': 'বাংলাদেশ নির্বাচন', 'detected_lang': 'bn', 'normalized': 'বাংলাদেশ নির্বাচন', 'q_en': 'Bangladesh selection', 'q_bn': 'বাংলাদেশ নির্বাচন', 'expansions': [], 'entities': [{'text': 'বাংলাদেশ', 'label': 'LOC'}], 'mapped_entities': [{'src': 'বাংলাদেশ', 'tgt': 'bangladesh', 'type': 'LOC'}], 'time_ms': {'total': 609, 'translation': 502, 'ner': 106}}
--------------------------------------------------------------------------------
{'query': 'Bangladesh election', 'detected_lang': 'en', 'normalized': 'bangladesh election', 'q_en': 'bangladesh election', 'q_bn': 'নিষেধাজ্ঞা নির্বাচন', 'expansions': ['vote', 'poll', 'ballot'], 'entities': [], 'mapped_entities': [], 'time_ms': {'total': 496, 'translation': 485, 'ner': 10}}
--------------------------------------------------------------------------------
{'query': 'ঢাকা traffic problem', 'detected_lang': 'bn', 'normalized': 'ঢাকা traffic problem', 'q_en': 'Dhaka', 'q_bn': 'ঢাকা traffic problem', 'expansions': [], 'entities': [{'text': 'ঢা

In [ ]:
LATIN_RANGE = re.compile(r'[A-Za-z]')
BN_RANGE = re.compile(r'[\u0980-\u09FF]')

def is_mixed_script(text: str) -> bool:
    return bool(BN_RANGE.search(text)) and bool(LATIN_RANGE.search(text))

def split_mixed(text: str):
    bn_part = " ".join(re.findall(r'[\u0980-\u09FF]+', text))
    en_part = " ".join(re.findall(r'[A-Za-z]+', text))
    return bn_part.strip(), en_part.strip()


In [ ]:
def postprocess_bn_to_en(q_bn: str, q_en: str) -> str:
    if "নির্বাচন" in q_bn:
        q_en = q_en.replace("selection", "election")
    return q_en

def postprocess_en_to_bn(q_en: str, q_bn: str) -> str:
    # If election appears, ensure Bangla contains নির্বাচন (not নিষেধাজ্ঞা নির্বাচন)
    if "election" in q_en and "নির্বাচন" not in q_bn:
        q_bn = q_bn.replace("নিষেধাজ্ঞা", "").strip()
        if "নির্বাচন" not in q_bn:
            q_bn = f"{q_bn} নির্বাচন".strip()
    # If results appears, add ফলাফল if missing
    if "results" in q_en and "ফলাফল" not in q_bn:
        q_bn = f"{q_bn} ফলাফল".strip()
    return q_bn


#Testing

In [ ]:
import re
import time
from typing import Dict, List, Tuple, Optional
from transformers import MarianMTModel, MarianTokenizer

# -------------------------
# Language Detection
# -------------------------
BN_RANGE = re.compile(r"[\u0980-\u09FF]")

def detect_lang(query: str) -> str:
    """Detect if query is Bangla or English"""
    return "bn" if BN_RANGE.search(query or "") else "en"


# -------------------------
# Stopwords (Optional but Recommended)
# -------------------------
STOPWORDS_EN = {
    "the", "a", "an", "is", "are", "was", "were", "in", "on", "at",
    "to", "for", "of", "and", "or", "but", "with", "from", "by"
}

STOPWORDS_BN = {
    "এবং", "বা", "কিন্তু", "যা", "যে", "এই", "সেই", "ও", "তে", "থেকে",
    "দিয়ে", "হয়", "হয়েছে", "করা", "করে", "একটি", "কোনো", "আর"
}

def remove_stopwords(query: str, lang: str) -> str:
    """Remove stopwords (optional preprocessing)"""
    stopwords = STOPWORDS_BN if lang == "bn" else STOPWORDS_EN
    words = query.split()
    filtered = [w for w in words if w.lower() not in stopwords]
    return " ".join(filtered) if filtered else query  # fallback to original if all removed


# -------------------------
# Normalization
# -------------------------
def normalize_query(query: str, lang: str, remove_stops: bool = False) -> str:
    """Normalize query: trim, lowercase (EN only), optional stopword removal"""
    q = " ".join((query or "").strip().split())
    if lang == "en":
        q = q.lower()
    if remove_stops:
        q = remove_stopwords(q, lang)
    return q


# -------------------------
# Translation (OPUS-MT)
# -------------------------
class TranslatorOPUS:
    """
    OPUS-MT translator with caching
    - bn → en: Helsinki-NLP/opus-mt-bn-en
    - en → bn: Helsinki-NLP/opus-mt-en-iir with >>ben<< token
    """
    def __init__(self, max_length: int = 64):
        self.cache = {}
        self.max_length = max_length

    def _load(self, model_name: str) -> Tuple[MarianTokenizer, MarianMTModel]:
        if model_name not in self.cache:
            tok = MarianTokenizer.from_pretrained(model_name)
            mdl = MarianMTModel.from_pretrained(model_name)
            self.cache[model_name] = (tok, mdl)
        return self.cache[model_name]

    def _generate(self, model_name: str, text: str) -> str:
        tok, mdl = self._load(model_name)
        batch = tok([text], return_tensors="pt", truncation=True)
        gen = mdl.generate(**batch, max_length=self.max_length)
        return tok.decode(gen[0], skip_special_tokens=True).strip()

    def translate_bn_to_en(self, text: str) -> str:
        return self._generate("Helsinki-NLP/opus-mt-bn-en", text)

    def translate_en_to_bn(self, text: str) -> str:
        text_with_lang = f">>ben<< {text}"
        out = self._generate("Helsinki-NLP/opus-mt-en-iir", text_with_lang)
        return out.replace(">>ben<<", "").strip()

    def translate(self, text: str, src: str, tgt: str) -> str:
        if src == "bn" and tgt == "en":
            return self.translate_bn_to_en(text)
        if src == "en" and tgt == "bn":
            return self.translate_en_to_bn(text)
        raise ValueError(f"Unsupported translation: {src}→{tgt}")


# -------------------------
# Query Expansion - IMPROVED
# -------------------------
EXPAND_EN = {
    # Politics
    "election": ["vote", "poll", "ballot", "voting"],
    "government": ["administration", "authority", "regime"],
    "minister": ["secretary", "official"],
    "parliament": ["legislature", "assembly"],

    # Education
    "education": ["school", "university", "learning", "teaching"],
    "student": ["pupil", "learner"],
    "teacher": ["instructor", "educator"],

    # Economy
    "economy": ["finance", "inflation", "market", "trade"],
    "business": ["commerce", "trade", "industry"],
    "price": ["cost", "rate", "tariff"],

    # Sports
    "cricket": ["test", "odi", "t20"],
    "football": ["soccer", "match"],
    "player": ["athlete", "sportsman"],

    # Common
    "news": ["report", "article", "story"],
    "death": ["died", "killed", "deceased"],
    "attack": ["assault", "strike"],
}

EXPAND_BN = {
    "নির্বাচন": ["ভোট", "পোল", "নির্বাচনী"],
    "সরকার": ["প্রশাসন", "কর্তৃপক্ষ"],
    "শিক্ষা": ["স্কুল", "বিশ্ববিদ্যালয়", "পড়াশোনা"],
    "অর্থনীতি": ["ফিন্যান্স", "বাজার"],
    "ক্রিকেট": ["টেস্ট", "ওডিআই", "টি-টুয়েন্টি"],
}

def expand_query(q: str, lang: str) -> List[str]:
    """Expand query with synonyms and related terms"""
    expand_dict = EXPAND_BN if lang == "bn" else EXPAND_EN

    terms = (q or "").lower().split() if lang == "en" else (q or "").split()
    expanded = []

    for term in terms:
        if term in expand_dict:
            expanded.extend(expand_dict[term])

    # Remove duplicates while preserving order
    return list(dict.fromkeys(expanded))


# -------------------------
# Named Entity Mapping - IMPROVED
# -------------------------
NE_EN2BN = {
    # Places
    "bangladesh": "বাংলাদেশ",
    "dhaka": "ঢাকা",
    "chittagong": "চট্টগ্রাম",
    "sylhet": "সিলেট",
    "rajshahi": "রাজশাহী",
    "khulna": "খুলনা",
    "cox's bazar": "কক্সবাজার",

    # Political Figures
    "sheikh hasina": "শেখ হাসিনা",
    "khaleda zia": "খালেদা জিয়া",
    "muhammad yunus": "মুহাম্মদ ইউনূস",

    # Organizations
    "awami league": "আওয়ামী লীগ",
    "bnp": "বিএনপি",
    "jamaat": "জামায়াত",

    # Institutions
    "dhaka university": "ঢাকা বিশ্ববিদ্যালয়",
    "buet": "বুয়েট",
}

NE_BN2EN = {v: k for k, v in NE_EN2BN.items()}

def map_named_entities(q: str, lang: str) -> List[Dict[str, str]]:
    """Map named entities between languages"""
    mapped = []
    q_normalized = q.lower() if lang == "en" else q

    if lang == "en":
        for en_term, bn_term in NE_EN2BN.items():
            if en_term in q_normalized:
                mapped.append({"src": en_term, "tgt": bn_term})
    else:
        for bn_term, en_term in NE_BN2EN.items():
            if bn_term in q_normalized:
                mapped.append({"src": bn_term, "tgt": en_term})

    return mapped


# -------------------------
# Translation Post-processing
# -------------------------
def postprocess_translation(q_original: str, q_translated: str, src_lang: str) -> str:
    """Fix common translation errors"""
    if src_lang == "bn":
        # নির্বাচন should be "election" not "selection"
        if "নির্বাচন" in q_original:
            q_translated = q_translated.replace("selection", "election")
        # ক্রিকেট should stay "cricket" not "criket"
        if "ক্রিকেট" in q_original:
            q_translated = q_translated.replace("criket", "cricket")

    return q_translated


class QueryProcessor:
    def __init__(self, use_stopwords: bool = False):
        """
        Initialize query processor

        Args:
            use_stopwords: Whether to remove stopwords during normalization
        """
        self.translator = TranslatorOPUS()
        self.use_stopwords = use_stopwords

    def process(self, raw_query: str) -> Dict:
        """
        Process query through complete pipeline

        Returns dict with:
            - original, detected_lang, normalized
            - q_en, q_bn (both languages always available)
            - expansions, named_entity_mappings
            - timing information
        """
        t0 = time.time()

        # Step 1: Language Detection
        lang = detect_lang(raw_query)

        # Step 2: Normalization
        q_norm = normalize_query(raw_query, lang, self.use_stopwords)

        # Step 3: Translation (with error handling)
        try:
            if lang == "bn":
                q_bn = q_norm
                q_en = self.translator.translate(q_norm, "bn", "en")
                q_en = postprocess_translation(q_bn, q_en, "bn")
            else:
                q_en = q_norm
                q_bn = self.translator.translate(q_norm, "en", "bn")
        except Exception as e:
            # Fallback: no translation but don't crash
            q_en = q_norm if lang == "en" else raw_query
            q_bn = q_norm if lang == "bn" else raw_query
            print(f"⚠️ Translation failed: {e}")

        # Step 4: Query Expansion
        expansions = expand_query(q_norm, lang)

        # Step 5: Named Entity Mapping
        ne_mappings = map_named_entities(q_norm, lang)

        total_ms = int((time.time() - t0) * 1000)

        return {
            "original": raw_query,
            "detected_lang": lang,
            "normalized": q_norm,
            "q_en": q_en,
            "q_bn": q_bn,
            "expansions": expansions,
            "named_entity_mappings": ne_mappings,
            "module_b_time_ms": total_ms,
        }


if __name__ == "__main__":
    processor = QueryProcessor(use_stopwords=False)

    test_queries = [
        "Bangladesh election results 2024",
        "ঢাকায় ক্রিকেট খেলা",
        "Sheikh Hasina news",
        "শিক্ষা ব্যবস্থা সংস্কার",
    ]

    print("="*60)
    print("MODULE B - QUERY PROCESSING DEMO")
    print("="*60)

    for query in test_queries:
        print(f"\n📝 Query: {query}")
        result = processor.process(query)

        print(f"   Language: {result['detected_lang']}")
        print(f"   Normalized: {result['normalized']}")
        print(f"   English: {result['q_en']}")
        print(f"   Bangla: {result['q_bn']}")

        if result['expansions']:
            print(f"   Expansions: {result['expansions']}")

        if result['named_entity_mappings']:
            print(f"   NE Mappings: {result['named_entity_mappings']}")

        print(f"   Time: {result['module_b_time_ms']}ms")

MODULE B - QUERY PROCESSING DEMO

📝 Query: Bangladesh election results 2024
   Language: en
   Normalized: bangladesh election results 2024
   English: bangladesh election results 2024
   Bangla: নিষেধাজ্ঞার নির্বাচনের ফলাফল ২০২৪
   Expansions: ['vote', 'poll', 'ballot', 'voting']
   NE Mappings: [{'src': 'bangladesh', 'tgt': 'বাংলাদেশ'}]
   Time: 2120ms

📝 Query: ঢাকায় ক্রিকেট খেলা
   Language: bn
   Normalized: ঢাকায় ক্রিকেট খেলা
   English: Cricket Games in Dhaka
   Bangla: ঢাকায় ক্রিকেট খেলা
   Expansions: ['টেস্ট', 'ওডিআই', 'টি-টুয়েন্টি']
   NE Mappings: [{'src': 'ঢাকা', 'tgt': 'dhaka'}]
   Time: 2146ms

📝 Query: Sheikh Hasina news
   Language: en
   Normalized: sheikh hasina news
   English: sheikh hasina news
   Bangla: শেখ হ্যাসিনা সংবাদ
   Expansions: ['report', 'article', 'story']
   NE Mappings: [{'src': 'sheikh hasina', 'tgt': 'শেখ হাসিনা'}]
   Time: 534ms

📝 Query: শিক্ষা ব্যবস্থা সংস্কার
   Language: bn
   Normalized: শিক্ষা ব্যবস্থা সংস্কার
   English: How can we sho